In [19]:
import DCCRN
import torch
from asteroid.data import DNSDataset
import os
import time
import torch.nn as nn
import shutil
import numpy as np
from framework import ABF, MultiResolutionSTFTLoss, SPKDLoss, build_review_kd, ReviewKD
import asteroid
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import config as cfg
from tools_for_model import near_avg_index, max_index, min_index, Bar, cal_pesq, cal_stoi
from dataloader import create_dataloader
import inspect


In [20]:
model = DCCRN.DCCRN(rnn_units=cfg.rnn_units, masking_mode=cfg.masking_mode, use_clstm=cfg.use_clstm,
                  kernel_num=cfg.kernel_num)
#data_set =  DNSDataset("/root/NTH_student/Speech_Enhancement_new/asteroid/egs/dns_challenge/baseline/data")
train_dataset =  DNSDataset("/root/NTH_student/test_loader")
train_loader = create_dataloader(mode='train',dataset=train_dataset)

In [21]:
# tạo sample để input mode. cái này random đại thui
# input_size = model.stft.weight.size(1)
# tạo cái batch có n= 5 đi
# input = torch.randn(64, input_size)

In [22]:
# Tạo list chứa các feature map do encoder output ra 
# dùng list vì lát nữa feature map của lstm với decoder bỏ vô đây luôn. ở đây demo cái encoder thui nha. 
feature_maps = []
# Tạo hàm hook, hàm này sẽ thực thi khi encoder thực hiện forward.
# output của encoder sẽ được append vào trong encoder_feature_map(một biến toàn cục)
def encoder_hook(module, input, output):
    feature_maps.append(output)
# đăng kí forward hook. kiểu "ê khi nào encoder thực hiện foward xong thì chạy hàm encoder_hook nha"
model.encoder[-1].register_forward_hook(encoder_hook)

In [23]:
def decoder_hook(module, input, output):
    feature_maps.append(output)

model.decoder[-1].register_forward_hook(decoder_hook)

In [24]:
# def enhance_hook(module, input, output):
#     feature_maps.append(output)

# model.enhance.register_forward_hook(enhance_hook)

In [25]:
X,Y = 0,0
i = 0
for x,y,_ in Bar(train_loader):
    if i == 1: break
    X=x
    Y=y
    i += 1

try :
    model(X)
    print('au de')
except:
    print('failed roihuhu')

 16/600: [>.................................................] - ETA 0.8sau de


In [26]:
len(feature_maps)

2

In [27]:
feature_maps[0].shape #[64, 256, 4, 483]


torch.Size([8, 256, 4, 483])

In [28]:
feature_maps[1].shape

torch.Size([8, 2, 256, 484])

In [29]:
in_channels = [32, 64, 128, 256, 256, 256]
out_channels = [32, 64, 128, 256, 256, 256]

# shapes = [1,4,8,16,32]
# out_shapes = [1,4,8,16,32]
shapes = [4,4,4]
out_shapes = [4,4,4]
abfs = nn.ModuleList()
mid_channel = min(512, in_channels[-1])
for idx, in_channel in enumerate(in_channels):
    abfs.append(ABF(in_channel, mid_channel, out_channels[idx], idx < len(in_channels)-1))
abfs = abfs[::-1]

In [30]:
student_features = model(X,is_feat=True)

In [31]:
logit = student_features
x = feature_maps[0] #encoder
results = []
out_features, res_features = abfs[0](x, out_shape=out_shapes[0])
results.append(out_features)
for abf, shape, out_shape in zip(abfs[1:], shapes[1:], out_shapes[1:]):
        out_features, res_features = abf(x, res_features, shape, out_shape)
results.insert(0, out_features)

In [32]:
results

[tensor([[[[-1.3222e+00, -8.9411e-01, -7.2742e-01, -2.8366e+00],
           [-1.2539e+00,  2.8833e+00, -3.6291e-02,  6.6587e-01],
           [ 8.1623e-02, -7.6165e+00, -2.3029e+00, -1.9873e+00],
           [ 2.1454e-01,  6.7703e-01,  4.2594e-01,  2.0992e-01]],
 
          [[ 2.1896e+00,  5.2855e+00, -1.5124e+00, -3.8074e+00],
           [-6.6654e-01, -2.5278e+00, -1.2095e+00,  5.2766e+00],
           [ 2.8943e-01,  3.7082e-01,  2.2154e+00, -2.0367e+00],
           [ 1.3596e-02,  2.8615e-01,  4.1438e-01, -6.0898e-01]],
 
          [[ 1.2320e+00,  6.2861e-01,  2.6034e+00, -4.9569e+00],
           [ 8.7635e-01,  9.5193e-01,  1.1344e+00, -1.7390e+00],
           [-1.2537e+00, -1.8615e+00, -5.2041e+00,  1.4772e+00],
           [ 3.4433e-04,  3.1820e-01, -2.6376e-01, -5.7935e-01]],
 
          ...,
 
          [[ 6.1556e-01, -2.9198e+00,  2.2692e+00,  3.3637e+00],
           [-1.0380e+00, -1.0642e+00, -3.1786e+00,  3.5835e+00],
           [ 4.7406e-01,  1.4743e+00,  5.4528e+00, -5.3188e+00],

In [33]:
in_channels = [32, 64, 128, 256, 64, 2]
out_channels = [32, 64, 128, 256, 64, 256]

# shapes = [1,4,8,16,32]
# out_shapes = [1,4,8,16,32]
shapes = [1,4,4,4,4]
#out_shapes = [1,4,4]
out_shapes = [4,4,4,4,4]
abfs = nn.ModuleList()
mid_channel = min(512, in_channels[-1])
for idx, in_channel in enumerate(in_channels):
    abfs.append(ABF(in_channel, mid_channel, out_channels[idx], idx < len(in_channels)-1))
abfs = abfs[::-1]

In [34]:
student_features = model(X,is_feat=True)
logit = student_features[0]
x = feature_maps[2]
results = []
out_features, res_features = abfs[0](x, out_shape=out_shapes[0])
results.append(out_features)
for abf, shape, out_shape in zip(abfs[:1], shapes[:1], out_shapes[:1]):
        out_features, res_features = abf(x, res_features, shape, out_shape)
        results.insert(0, out_features)
results

RuntimeError: Given groups=1, weight of size [2, 2, 1, 1], expected input[8, 256, 4, 483] to have 2 channels, but got 256 channels instead

In [ ]:
results[0].shape

torch.Size([64, 256, 256, 4])

In [ ]:
feature_maps[2].shape

torch.Size([64, 2, 256, 4])

In [ ]:
feature_maps[1][0].shape

torch.Size([3, 64, 512])

In [ ]:
feature_maps[1][1].shape

torch.Size([3, 64, 512])

In [ ]:
feature_maps.clear()